# Sycomp Onboarding - Session 1

0. Platform Architecture + K8s Background

1. Iguazio Overview

2. Code to Microservice

3. More on MLRun Functions

## 0. Platform Architecture + Docker/K8s Background

![Docker](img/docker.png)
- Docker
    - A container is a standard unit of software that packages up code and all its dependencies so the application runs quickly and reliably from one computing environment to another.
    - A Docker container image is a lightweight, standalone, executable package of software that includes everything needed to run an application: code, runtime, system tools, system libraries and settings.

![K8s](img/k8s.png)
- Kubernetes (also known as K8s)
    - Open-source system for automating deployment, scaling, and management of containerized applications.
    - One stop shop - manages compute, networking, storage, permissions, scaling, etc.

- Platform Architecture
![Platform Architecture](img/platform_architecture.png)

## 1. Iguazio Overview
- UI Tour
- Project Management
- Managed Services

## 2. Code to Microservice
- Job Example (Python)
- Job Example (Notebook)
- Nuclio Example

- Package Python code into microservice to run on K8s

- ```python
fn = code_to_function(
    name="test",
    project="my_project",
    filename="test.py", # .py or .ipynb
    kind="job",
    image="mlrun/mlrun",
    handler="my_func"
)
fn.run()
```

- Add hooks for experiment tracking and more

- Experiment tracking
    ```python
    context.log_model()
    context.log_dataset()
    context.log_result()
    ```

- Pass inputs/parameters
    ```python
    context.get_param("batch_size")
    context.get_input("training_set")
    ```

- Logging
    ```python
    context.logger.info("Something happened")
    context.logger.error("Something went wrong")
    ```

### Job Example - Python

In [1]:
from mlrun import code_to_function

job = code_to_function(
    name="my_job",
    project="onboarding",
    filename="assets/job.py",
    kind="job",
    image="mlrun/mlrun",
    handler="my_func"
)
job.run(params={"value" : 123}, local=True)

> 2021-09-15 18:01:15,331 [info] starting run my-job-my_func uid=d01d9cbd2e704ca597e6d02872156e9a DB=http://mlrun-api:8080
> 2021-09-15 18:01:15,506 [info] Logging value
> 2021-09-15 18:01:15,518 [info] Logging CSV
> 2021-09-15 18:01:15,756 [info] Logging model


Converting input from bool to <class 'numpy.uint8'> for compatibility.


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
onboarding,...72156e9a,0,Sep 15 18:01:15,completed,my-job-my_func,v3io_user=nickkind=owner=nickhost=jupyter-nick-55654b7c59-q5zvd,,value=123,value=123,my_csvmy_model


to track results use .show() or .logs() or in CLI: 
!mlrun get run d01d9cbd2e704ca597e6d02872156e9a --project onboarding , !mlrun logs d01d9cbd2e704ca597e6d02872156e9a --project onboarding
> 2021-09-15 18:01:15,961 [info] run executed, status=completed


### Job Example - Notebook

In [2]:
from mlrun import auto_mount

job = code_to_function(
    name="my_notebook",
    project="onboarding",
    filename="assets/notebook.ipynb",
    kind="job",
    image="mlrun/mlrun",
    handler="main"
)
job.apply(auto_mount())
job.run(params={"test_size" : 0.3, "gamma" : 0.001, "C" : 100}, local=False)

> 2021-09-15 18:01:41,529 [info] starting run my-notebook-main uid=a50597f6f1a34f22b3de71f6e004da3d DB=http://mlrun-api:8080
> 2021-09-15 18:01:41,706 [info] Job is running in the background, pod: my-notebook-main-p7jxf
> 2021-09-15 18:01:46,900 [info] Getting data
> 2021-09-15 18:01:46,904 [info] Training model
> 2021-09-15 18:01:46,906 [info] Evaluating model
> 2021-09-15 18:01:46,908 [info] Logging datasets
> 2021-09-15 18:01:47,190 [info] Logging model
> 2021-09-15 18:01:47,318 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
onboarding,...e004da3d,0,Sep 15 18:01:46,completed,my-notebook-main,v3io_user=nickkind=jobowner=nickhost=my-notebook-main-p7jxf,,test_size=0.3gamma=0.001C=100,accuracy=1.0,X_trainX_testy_trainy_testnotebook_model


to track results use .show() or .logs() or in CLI: 
!mlrun get run a50597f6f1a34f22b3de71f6e004da3d --project onboarding , !mlrun logs a50597f6f1a34f22b3de71f6e004da3d --project onboarding
> 2021-09-15 18:01:47,920 [info] run executed, status=completed


### Real-Time (Nuclio) Example

In [3]:
rt = code_to_function(
    name="my_rt",
    project="onboarding",
    filename="assets/rt.py",
    kind="nuclio",
    image="mlrun/mlrun",
    handler="handler"
)
rt.deploy()

> 2021-09-15 18:01:52,385 [info] Starting remote function deploy
2021-09-15 18:01:52  (info) Deploying function
2021-09-15 18:01:52  (info) Building
2021-09-15 18:01:52  (info) Staging files and preparing base images
2021-09-15 18:01:52  (info) Building processor image
2021-09-15 18:01:53  (info) Build complete
2021-09-15 18:01:57  (info) Function deploy complete
> 2021-09-15 18:01:58,139 [info] function deployed, address=default-tenant.app.us-sales.iguazio-cd0.com:31103


'http://default-tenant.app.us-sales.iguazio-cd0.com:31103'

In [4]:
%%time
rt.invoke(path="/")

CPU times: user 8.53 ms, sys: 638 µs, total: 9.17 ms
Wall time: 25.3 ms


{'result': '0.14122883413563148'}

## 3. More on MLRun Functions
- Basic configuration - Limits/requests
- Import/export
- Marketplace integration

### Basic Configuration - Limits/Requests

https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/#requests-and-limits

In [5]:
from mlrun import auto_mount

job = code_to_function(
    name="my_notebook",
    project="onboarding",
    filename="assets/notebook.ipynb",
    kind="job",
    image="mlrun/mlrun",
    handler="main"
)
job.apply(auto_mount())
job.with_requests(cpu=2)
job.with_limits(cpu=2)
job.run(params={"test_size" : 0.3, "gamma" : 0.001, "C" : 100}, local=False)

> 2021-09-15 18:02:30,200 [info] starting run my-notebook-main uid=64c54abfb4154995b979d6018c00cb00 DB=http://mlrun-api:8080
> 2021-09-15 18:02:30,387 [info] Job is running in the background, pod: my-notebook-main-5tpjn
> 2021-09-15 18:02:35,569 [info] Getting data
> 2021-09-15 18:02:35,573 [info] Training model
> 2021-09-15 18:02:35,575 [info] Evaluating model
> 2021-09-15 18:02:35,577 [info] Logging datasets
> 2021-09-15 18:02:36,040 [info] Logging model
> 2021-09-15 18:02:36,207 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
onboarding,...8c00cb00,0,Sep 15 18:02:35,completed,my-notebook-main,v3io_user=nickkind=jobowner=nickhost=my-notebook-main-5tpjn,,test_size=0.3gamma=0.001C=100,accuracy=0.9333333333333333,X_trainX_testy_trainy_testnotebook_model


to track results use .show() or .logs() or in CLI: 
!mlrun get run 64c54abfb4154995b979d6018c00cb00 --project onboarding , !mlrun logs 64c54abfb4154995b979d6018c00cb00 --project onboarding
> 2021-09-15 18:02:36,537 [info] run executed, status=completed


### Export

In [6]:
# print(rt.to_yaml())

In [7]:
rt.export()

> 2021-09-15 18:02:47,541 [info] function spec saved to path: function.yaml


### Import (File)

In [8]:
from mlrun import import_function

In [9]:
new_rt = import_function("function.yaml")

In [10]:
# print(new_rt.to_yaml())

### Import (Marketplace)

In [11]:
# Reaches out to https://github.com/mlrun/functions (customizable)
fn = import_function("hub://describe")

In [12]:
# print(fn.to_yaml())